This notebook is inspired by 
https://neuronaldynamics-exercises.readthedocs.io/en/latest/exercises/hopfield-network.html#getting-started


neurodynex3 is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License 2.0 as published by the Free Software Foundation. You should have received a copy of the GNU General Public License along with the repository. If not, see http://www.gnu.org/licenses/.

Should you reuse and publish the code for your own purposes, please point to the webpage http://neuronaldynamics.epfl.ch or cite the book: Wulfram Gerstner, Werner M. Kistler, Richard Naud, and Liam Paninski. Neuronal Dynamics: From Single Neurons to Networks and Models of Cognition. Cambridge University Press, 2014.


Useful functions for this exercise:



In [ ]:
%matplotlib inline
import brian2 as b2
import matplotlib.pyplot as plt
import numpy as np
from neurodynex3.hopfield_network import network, pattern_tools, plot_tools

Purpose of the exercise is to test the robustness of a Hopfield network encoding for a list of letters.

First part:
- what is the theoretical capacity of a Hopfield network of $N=100$ units?
- Consider a list of letters: ['A', 'B', 'C', 'F', 'X']. Each letter is a memory pattern.
  - Store the list
  - Show that the patterns equilibirum points
  - Show if they are attactors.

In [ ]:
# load the alphabet dictionary
abc_dictionary =pattern_tools.load_alphabet()

# define the letters to be stored in the network
letter_list = ['A', 'B', 'C', 'F', 'X']

print("the letters to be stored in the network are: {}".format(letter_list))

## each pattern has the same shape of (10, 10)
pattern_shape = abc_dictionary[letter_list[0]].shape
## create an instance of the class HopfieldNetwork with the number of neurons
# equal to the number of pixels in the pattern
# (10*10 = 100)
hopfield_net = network.HopfieldNetwork(nr_neurons = pattern_shape[0]*pattern_shape[1])

# # create a list using Pythons List Comprehension syntax:
pattern_list = [abc_dictionary[key] for key in letter_list ]
plot_tools.plot_pattern_list(pattern_list)

# # store the patterns
hopfield_net.store_patterns(pattern_list)

In [ ]:
# check if all patterns are equilibrium states: 
i = 0
for pattern in pattern_list:
    hopfield_net.set_state_from_pattern(pattern)
    states = hopfield_net.run_with_monitoring(nr_steps=10)
    states_as_patterns = pattern_tools.reshape_patterns(states, pattern.shape)
    plot_tools.plot_state_sequence_and_overlap(
        states_as_patterns, pattern_list, reference_idx=i, suptitle="Network dynamics for {}".format(letter_list[i]))
    i += 1


In [ ]:
# check if all patterns are attractors states: 
# start from the same noise in the next run
np.random.seed(123)
# start from a noisy pattern: 
i = 0
for pattern in pattern_list:
    noisy_init_state = pattern_tools.get_noisy_copy(pattern, noise_level=0.2)
    hopfield_net.set_state_from_pattern(noisy_init_state)
    states = hopfield_net.run_with_monitoring(nr_steps=6)
    states_as_patterns = pattern_tools.reshape_patterns(states, pattern.shape)
    plot_tools.plot_state_sequence_and_overlap(
        states_as_patterns, pattern_list, reference_idx=i, suptitle="Network dynamics for {}".format(letter_list[i]))
    i += 1

In [ ]:
# compute similarity between patterns
print("similarity between patterns:")
for i in range(len(pattern_list)):
    print("Similarity between {} and others: ".format(letter_list[i]), pattern_tools.compute_overlap_list(pattern_list[i], pattern_list))

Second part:
Consider now two different Hopfield networks. the first one stores ['A', 'B', 'C', 'F', 'E'] and the second ['A', 'B', 'C', 'F', 'Y']. Study pattern completion in these networks by starting from slightly perturbed initial conditions. Which network is more stable? Why? Can you propose a metric to asses robustness?

Bonus: 
Make a guess of how many letters the network can store. Then create a (random) set of letters. Check if all letters of your list are fixed points under the network dynamics. Explain the discrepancy between the theoretical capacity C (computed above) and your observation.